# STAT 760 Homework 6
## Lilly Peacor

The below code will be a function that takes input of a dataset, a list where each entry is the number of neurons in that layer, number of entries per batch, number of epochs, accuracy level to stop early, and the activation functions for hidden layers and the output layer.

In [ ]:
#@title getting dataset from kaggle
from google.colab import files
#files.upload()
#!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import math
import sys
import random

def NN(X, Y, learning_rate = 0.01, epochs = 10, min_acc = 0.1, s1 = "relu", s2 = "id", layers = [2, 2, 2], batch_size = 100, print_diag = True, print_res = True):
  """
  This function takes input of a dataset, assuming normalization and factorization has already occured.
  The inputs are the input dataset, the output dataset, learning rate, the maximum number of epochs, the accuracy that
    would cause the function to halt training, the activation function for all hidden layers, the
    activation function for the output layer, a list of the number of neurons to include in each hidden layer -
    this also shows the number of hidden layers via len(layers), the batch size for training, and whether or not to print the diagnostics

  This function is not intended to surpass existing neural network functions in computing power or accuracy, but instead to give
    better insight into how a fully-connected feed forward neural network is trained.

  The function has several control points to ensure inputs are correct for the training process. If the inputs to
    the function fail any checks, an error message will print to the terminal and the function will return null.
    The checks include ensuring input and output are of same size, the batch size is not larger than the training data,
    and if the selected activation functions are valid options.

  The function will output a dictionary that contains the weight matrix 'w', the bias vectors 'b', the accuracy on train data 'train_acc',
    the accuracy on test data 'test_acc', and the model output on the entire input dataset 'output'.

  """

  #check activation functions
  if s1 not in ['leaky_relu', 'relu', 'sigmoid', 'tanh'] or s2 not in ['leaky_relu', 'relu', 'sigmoid', 'tanh', 'id']:
    print("Invalid activation function selected. Please choose from ['leaky_relu', 'relu', 'sigmoid', 'tanh', 'id]")
    return None

  #check dataset lengths
  if len(X) != len(Y):
    print("Input and output datasets are not the same size.\n")
    return None

  #split into test and train sets using 70-30 split
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=2525)

  #check if batch size is bigger than the size of the dataset. If it is, return null and print error message
  if batch_size > len(X_train):
    print("Batch size cannot be bigger than the size of the dataset.\n")
    return None

  if print_diag:
    print("There are", len(X_train), "training samples and", len(X_test), "testing samples.\n")

  #defining different activation functions and their derivatives
  def relu(x):
    return np.maximum(0, x)

  def relu_grad(x):
    if x > 0:
      return 1
    elif x == 0:
      return 0.5
    else:
      return 0

  def leaky_relu(x, alpha = 0.01):
    return np.maximum(alpha * x, x)

  def leaky_relu_grad(x, alpha = 0.01):
    if x > 0:
      return 1
    elif x == 0:
      return (1 + alpha) / 2
    else:
      return alpha

  def sigmoid(x):
    return 1 / (1 + math.exp(-x))

  def sigmoid_grad(x):
    return math.exp(-x) / (1 + math.exp(-x))**2

  def tanh(x):
    return math.tanh(x)

  def tanh_grad(x):
    return 1 / math.cosh(x)**2

  def id(x):
    return x

  def id_grad(x):
    return 1

  #define the activation functions to use in calculations
  def sig1(x):
    if s1 == "relu":
      return relu(x)
    elif s1 == "leaky_relu":
      return leaky_relu(x)
    elif s1 == "sigmoid":
      return sigmoid(x)
    elif s1 == "tanh":
      return tanh(x)
    elif s1 == "id":
      return id(x)

  def sig1_vect(x):
    return np.array([sig1(x[i]) for i in range(len(x))])

  def sig1_grad(x):
    if s1 == "relu":
      return relu_grad(x)
    elif s1 == "leaky_relu":
      return leaky_relu_grad(x)
    elif s1 == "sigmoid":
      return sigmoid_grad(x)
    elif s1 == "tanh":
      return tanh_grad(x)
    elif s1 == "id":
      return id_grad(x)

  def sig2(x):
    if s2 == "relu":
      return relu(x)
    elif s2 == "leaky_relu":
      return leaky_relu(x)
    elif s2 == "sigmoid":
      return sigmoid(x)
    elif s2 == "tanh":
      return tanh(x)
    elif s2 == "id":
      return id(x)

  def sig2_vect(x):
    return np.array([sig2(x[i]) for i in range(len(x))])

  def sig2_grad(x):
    if s2 == "relu":
      return relu_grad(x)
    elif s2 == "leaky_relu":
      return leaky_relu_grad(x)
    elif s2 == "sigmoid":
      return sigmoid_grad(x)
    elif s2 == "tanh":
      return tanh_grad(x)
    elif s2 == "id":
      return id_grad(x)

  #Some important numbers
  m0 = np.shape(X)[1] #number of input variables

  m_infty = 1 #number of output variables, this function currently only permits 1-d output

  h = len(layers) #number of hidden layers

  layers = [m0] + layers + [m_infty] #list of all layers and number of neurons

  if print_diag:
    for i, layer in enumerate(layers):
      print("Layer", i, "has", layer, "neuron(s).\n")

  #making and initializing the weight matrix
  ##We randomly assign weights
  ##There is a dummy row to make the "real" matrix index match with index in python
  def init_w(layers):
    weights = [list(np.zeros((1,1)))] #dummy weight matrix
    for i in range(1, h + 2):
      #each weight matrix is of size h_{i-1},h_i, weights assigned randomly
      weights.append([[random.random() for _ in range(layers[i - 1])] for _ in range(layers[i])])
    return weights

  weights = init_w(layers)

  def init_b(layers):
    biases = [list(np.zeros(1))] #dummy bias vector
    for i in range(1, h + 2):
      biases.append([0 for _ in range(layers[i])]) #initialize biases at 0
    return biases

  biases = init_b(layers)

  if print_diag:
    for i in range(1,h+2):
      print("layer",i)
      print("Weight matrix dimension is",layers[i],"x",layers[i-1],":")
      print(weights[i])
      print("Bias vector dimension is",layers[i],":")
      print(biases[i])
      print("\n")

  #Batching

  #list of all indices in Training set
  indices = [i for i in range(len(X_train))]

  #function to create a set of batches for each epoch, the batches will contain only indices of the original set of data
  def batch_gen(batch_size, indices):
    batches = [] #list of batches that will contain a list of indices for each batch
    random.shuffle(indices) #scramble the list of indices
    for i in range(math.floor(len(indices) / batch_size)):
      if i != math.floor(len(indices) / batch_size) - 1:
        batches.append(indices[i * batch_size : (i + 1) * batch_size])
      else:
        #final batch may be smaller than batch_size, this line avoids sampling out of bounds
        batches.append(indices[i * batch_size : ])
    return batches

  if print_diag:
    print("There are", math.ceil(len(X_train) / batch_size), "batches in each epoch.\n")

  #Function to calculate output of each layer
  def k_out(input, k, weights, biases):
    #if in input layer, don't do anything
    if k == 0:
      return input, input #2-D output for before and after activation function
    #initialize v with input layer
    v = np.array(input)
    #loop over each layer to arrive at output of layer k
    for i in range(1,k+1):
      #grab weight matrix and bias vector for this layer
      W = np.array(weights[i])
      b = np.array(biases[i])
      #compute u - Weights*v{i-1}+biases
      u= W@v + b
      #if the layer is not the output layer, use sig1, else use sig2
      if i < h+1:
        v = np.array(sig1_vect(u))
      else:
        v = np.array(sig2_vect(u))
      #if print_diag:
      #  print("Layer", i, "output is", v)
    return u, v

  #diagnostic print to ensure the above funciton is working as intended
  #if print_diag:
  #  #randomly generate some vector of size m0
  #  x = np.array([random.random() for _ in range(m0)])
  #  for k in range(h+2):
  #    print("Output of layer", k, "is", k_out(x, k, weights, biases)[1],"\n")

  #We will use a loss function of MSE here

  #We take the various partial derivatives of the loss function for gradient descent

  #w.r.t. v
  def dldv(B,k,i,s,weights,biases,layers):
    #This partial derivative is only used up to the last hidden layer, not the
    # output layer
    if k == h: #if in final hidden layer
      f1 = 2 * (k_out(X_train[i], h + 1, weights, biases)[1][0] - Y_train[i]) / len(B)
      f2 = sig2_grad(k_out(X_train[i], h + 1, weights, biases)[0][0])
      f3 = weights[h+1][0][s]
      return(f1 * f2 * f3)
    elif k < h: #if in hidden layer not the last one
      return(sum([dldv(B, k + 1, i, r, weights, biases, layers) * sig1_grad(k_out(X_train[i], k + 1, weights, biases)[0][r - 1]) * weights[k + 1][r][s] for r in range(layers[k + 1])]))
    else: #if not in hidden layers print error and return null
      print("k out of bounds for dldv, returning null")
      return None

  #w.r.t. w
  def dldw(B, k, r, s, weights, biases, layers):
    #output layer action
    if k == h+1:
      return(2 * sum([(k_out(X_train[i], h + 1, weights, biases)[1][0] - Y_train[i]) * sig2_grad(k_out(X_train[i], h + 1, weights, biases)[0][0]) * k_out(X_train[i], h, weights, biases)[1][s] for i in B]) / len(B))
    #hidden layer action
    if k < h+1:
      return(sum([dldv(B, k, i, r, weights, biases, layers) * sig1_grad(k_out(X_train[i], k, weights, biases)[0][r]) * k_out(X_train[i], k - 1, weights, biases)[1][s] for i in B]))
    #error for out of bounds k
    if k > h + 1:
      print("k out of bounds for dldw, returning null")
      return None

  #w.r.t. b
  def dldb(B, k, r, weights, biases, layers):
    #output layer action
    if k == h+1:
      return(sum([(k_out(X_train[i], h + 1, weights, biases)[1][0] - Y_train[i]) * sig2_grad(k_out(X_train[i], h + 1, weights, biases)[0][0]) for i in B]))
    #hidden layer action
    if k < h + 1:
      return(sum([dldv(B, k, i, r, weights, biases, layers) * sig1_grad(k_out(X_train[i], k, weights, biases)[0][r]) for i in B]))
    #error for out of bounds k
    if k > h + 1:
      print("k out of bounds for dldb, returning null")
      return None


  #The calculation of mse to be used to track overall accuracy on train and test data
  def MSE(X, Y, weights, biases):
    mse = 0
    for i in range(len(X)):
      mse += (k_out(X[i], h + 1, weights, biases)[1][0] - Y[i])**2
    return mse / len(X)

  #training the model via gradient descent
  if print_res:
    print("Model training in progress...")
    t1 = time.time()

  #lists to store accuracies throughout training, stores an entry each epoch
  ##These contain the MSE prior to training
  train_acc = [MSE(X_train, Y_train, weights, biases)]
  test_acc = [MSE(X_train, Y_train, weights, biases)]

  #count variable to track epochs
  epoch_num = 0

  #dummy variable to start off change in accuracy
  d_mse = min_acc + 10.0 #min_acc+10 ensure is larger than minimum accuracy change set

  #loop exits when either max number of epochs is reached or the change in mse is less than min_acc
  while epoch_num < epochs and d_mse > min_acc:
    #we re-batch each epoch randomly
    if print_diag:
      print(f"Epoch {epoch_num + 1} of {epochs}\n")
    for i, B in enumerate(batch_gen(batch_size, indices)):
      #loop over each batch
      if print_diag:
        print(f"Batch {i + 1} of {math.ceil(len(X_train) / batch_size)}\n")

      #work backwards through layers, starting at output layer ending at input layer
      for k in range(h + 1, 0, -1):
        #loop over each neuron in layer k
        for r in range(layers[k]):
          #adjust biases for each neuron, replacing old set of biases
          biases[k][r] -= learning_rate * dldb(B, k, r, weights, biases, layers)
          #loop over each input to neuron r in layer k
          for s in range(layers[k - 1]):
            #adjust weights for each input into neuron s
            weights[k][r][s] -= learning_rate * dldw(B, k, r, s, weights, biases, layers)

    #add new mse to the lists
    train_acc.append(MSE(X_train, Y_train, weights, biases))
    test_acc.append(MSE(X_test, Y_test, weights, biases))
    #alter change in accuracy
    d_mse = abs(train_acc[epoch_num + 1] - train_acc[epoch_num])
    #iterate the epoch number
    epoch_num += 1

  if print_res:
    t2 = time.time()
    print(f"Training took {round(t2 - t1, 2)} seconds to complete")
    print(f"Minimum change in MSE reached after {epoch_num - 1}")
    print(f"Final MSE for training data is {train_acc[-1]}")
    print(f"Final MSE for test data is {test_acc[-1]}")

  #Get model output on the full entered dataset
  NN_out = []
  for i in range(len(X)):
    NN_out.append(k_out(X[i], h + 1, weights, biases)[1][0])

  #create output dictionary
  output = {"w": weights, "b": biases, "train_acc": train_acc[-1], "test_acc": test_acc[-1], "Output": NN_out}

  return output




In [ ]:
#import data
import kagglehub
# Download latest version
path = kagglehub.dataset_download("akshaydattatraykhare/data-for-admission-in-the-university")

#print("Path to dataset files:", path)

data = pd.read_csv('/kaggle/input/data-for-admission-in-the-university/adm_data.csv')

#drop Serial No. column
data = data.drop('Serial No.', axis=1)

#inspect dataset
data.info()

#split to X and Y
X = data.drop('Chance of Admit ', axis=1)
Y = data['Chance of Admit ']

#normalize non-categorical variables
for col in X.columns:
  if col != 'Research' and col != 'University Rating':
    X[col] = (X[col] - X[col].mean()) / X[col].std()
  else:
    #OHE the categorical variables
    X = pd.get_dummies(X, columns=[col])

for col in X.columns:
  #convert to numeric for boolean variables
  if "University" in col or "Research" in col:
    X[col] = X[col].astype(int)

#reinspect X
X.info()
X.head()

#convert X and Y to numpy
X = np.array(X)
Y = np.array(Y)

#run the neural network function
#model_out = NN(X, Y, learning_rate = 0.01, epochs = 10, min_acc = 0.01, s1 = "relu", s2 = "id", layers = [2, 3, 4, 3], batch_size = 50, print_diag = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   GRE Score          400 non-null    int64  
 1   TOEFL Score        400 non-null    int64  
 2   University Rating  400 non-null    int64  
 3   SOP                400 non-null    float64
 4   LOR                400 non-null    float64
 5   CGPA               400 non-null    float64
 6   Research           400 non-null    int64  
 7   Chance of Admit    400 non-null    float64
dtypes: float64(4), int64(4)
memory usage: 25.1 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   GRE Score            400 non-null    float64
 1   TOEFL Score          400 non-null    float64
 2   SOP                  400 non-null    float64
 3   LOR  

In [ ]:
#Trying with varying the activation function since those settings appear to be great
model_out2 = NN(X, Y, learning_rate = 0.01, epochs = 10, min_acc = 0.005, s1 = "relu", s2 = "id", layers = [2, 4, 6, 3], batch_size = 50, print_diag = True)

There are 280 training samples and 120 testing samples.

Layer 0 has 12 neuron(s).

Layer 1 has 2 neuron(s).

Layer 2 has 4 neuron(s).

Layer 3 has 4 neuron(s).

Layer 4 has 3 neuron(s).

Layer 5 has 1 neuron(s).

layer 1
Weight matrix dimension is 2 x 12 :
[[0.5470201068802534, 0.9936752395350533, 0.7505703831298098, 0.6657461866604492, 0.7458062617943696, 0.5143085749336186, 0.5046618965039839, 0.1654390467734118, 0.6638407346289795, 0.17500578749862294, 0.45629361785028155, 0.37376238257657823], [0.4537065184929485, 0.41633745697557456, 0.16583883621423867, 0.5469990721656077, 0.6894522286705589, 0.050718394575320125, 0.651613813879537, 0.1989227667776362, 0.6458168194676914, 0.6118079811570232, 0.17151372489278094, 0.03229034116418472]]
Bias vector dimension is 2 :
[0, 0]


layer 2
Weight matrix dimension is 4 x 2 :
[[0.5599987166945318, 0.26998851135313895], [0.2625634618698178, 0.052945586284560364], [0.2874032363021888, 0.4561960974960525], [0.0027717731151478686, 0.413259495786

In [ ]:
#Printing the best model, just with no diagnostics so that results are shown
random.seed(2525)
model_out = NN(X, Y, learning_rate = 0.01, epochs = 10, min_acc = 0.001, s1 = "relu", s2 = "id", layers = [3, 4, 4, 3], batch_size = 50, print_diag = False)



Model training in progress...
Training took 651.5 seconds to complete
Minimum change in MSE reached after 4
Final MSE for training data is 0.006250618414247497
Final MSE for test data is 0.006643796622283756


The best MSE I could achieve was $0.006250618414247497$ on train data and $0.006643796622283756$ on test data. This used a hidden layer structure of 3, 4, 4, 3 neurons. The model would at times produce errors with higher numbers of neurons. This function used ReLU as the hidden layer activation function and identity as the output layer activation function. Batch size was 50, learning rate of $0.01$, and was able to reach a change in MSE of $0.001$ after only 4 epochs, never reaching the maximum epochs specified of 10. This function appears to not have significant overfitting due to accuracy being relatively similar between test and train data.